In [ ]:
import magmaforge
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import thermoengine as thermo


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:


def plot_phase_fractions(phase_frac_tbl, ax=None):
    mineral_frac_tbl = phase_frac_tbl.drop(columns=['Liquid','Water'])
    # mineral_frac_tbl.index = [int(iT-273) for iT in mineral_frac_tbl.index]
    mineral_frac_tbl.index = mineral_frac_tbl.index - 273
    fTOL = 1e-4
    cols = mineral_frac_tbl.max(axis=0)>fTOL

    mineral_frac_tbl.loc[::-1, cols].plot.bar(stacked=True, ax=ax)
    
    if ax is None:
        ax = plt.gca()
        
    ax.set_xlabel('Temp [C]')
    ax.set_ylabel('Mass Fraction')
    
def plot_magma_evolution(history):
    phase_frac_tbl = history.phase_frac_table
    
    liq_comp = history.liquid_comp_table
    liq_comp.index -= 273.15


    fig, ax = plt.subplots(nrows=3, sharex=False, squeeze=True, figsize=(5,10) )

    iax = ax[0]
    plot_phase_fractions(phase_frac_tbl, ax=iax)
    iax.set_xticklabels([])
    iax.set_xlabel('')

    iax=ax[1]
    liq_comp.plot(y=['MgO','FeO','Fe2O3','Al2O3','K2O','Na2O','H2O'], ax=iax).legend(loc='upper left')
    # iax.set_xlabel('Temp [C]')
    iax.set_ylabel('Magma Comp [wt%]')
    iax.set_xticklabels([])


    iax=ax[2]
    liq_comp.plot(y='SiO2', ax=iax, legend=True)
    iax.set_xlabel('Temp [C]')
    iax.set_ylabel('Magma Comp [wt%]')


In [ ]:
comp={
    'Thingmuli_dry': {
        'SiO2':49.91,
        'TiO2': 1.47,
        'Al2O3': 17.91,
        'Fe2O3': 2.45,
        'FeO':  7.02,
        'MnO':  0.0, #0.16
        'MgO':  6.62,
        'CaO':  10.02,
        'Na2O':  3.02,
        'K2O':  0.64,
        'P2O5':  0.2,
        'H2O':  0.0,
        'CO2': 1e-8,},
}
T0 = 1200.00+273+50

In [ ]:
# sys = magmaforge.System(comp=comp['Thingmuli_dry'], P=1e3, T0=T0, 
#                             O2_buffer='NNO', del_fO2=0, model_name='v1.2',
#                             frac_remains=.001)

# sys.crystallize(method='frac', fix_fO2=True, Tstep=5, calc_args={'stats':False, 'debug':0});


In [ ]:
# Rhyolite MELTS automatic method

sys = magmaforge.System(comp=comp['Thingmuli_dry'], P=1e3, T0=T0, 
                            O2_buffer=None, model_name='v1.2',
                            frac_remains=.001)
sys.crystallize(method='frac', Tstep=5, calc_args={'stats':False, 'debug':0});



In [ ]:

# Rhyolite MELTS manual method
modelDB = thermo.model.Database(liq_mod='v1.2')
Liq = modelDB.get_phase('Liq')

sys = magmaforge.System(comp=comp['Thingmuli_dry'], P=1e3, T0=T0, 
                            O2_buffer=None,  Liq=Liq, has_CO2=True,
                            frac_remains=.001)
sys.crystallize(method='frac', Tstep=5, calc_args={'stats':False, 'debug':0});

In [ ]:
plot_magma_evolution(sys.history)

In [ ]:
plot_magma_evolution(sys.history)

In [ ]:
def get_magma_evo_table(sys:magmaforge.System) -> pd.DataFrame:
    evo_tbl = sys.history.liquid_comp_table
    evo_tbl['rho'] = sys.history.get_liquid_densities()
    evo_tbl['rho_ref'] = sys.history.get_liquid_ref_densities()
    evo_tbl['T'] = sys.history.get_temps()
    mol_Fe3 = 2*evo_tbl['Fe2O3']/159.6922
    mol_Fe2 = evo_tbl['FeO']/71.8464
    
    evo_tbl['Fe_cat_ratio'] = mol_Fe3/mol_Fe2
    evo_tbl['log_Fe_cat_ratio'] = np.log10(evo_tbl['Fe_cat_ratio'])
    
    return evo_tbl

magma_evo = get_magma_evo_table(sys)

In [ ]:
sns.lineplot(magma_evo, x='T', y='log_Fe_cat_ratio')
plt.xlabel('Temp  [K]')
plt.ylabel('log$_{10}$ ( Fe$^{3+}$/ Fe$^{2+}$)')

In [ ]:
sns.lineplot(magma_evo, x='T', y='log_Fe_cat_ratio')
plt.xlabel('Temp  [K]')
plt.ylabel('log$_{10}$ ( Fe$^{3+}$/ Fe$^{2+}$)')